In [1]:
!pip install boto3

     |████████████████████████████████| 131 kB 7.6 MB/s 
     |████████████████████████████████| 79 kB 6.6 MB/s 
     |████████████████████████████████| 7.9 MB 44.6 MB/s 
     |████████████████████████████████| 138 kB 30.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.6 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
import boto3

BUCKET_NAME = 'kkudylambdazips' # replace with your bucket name

# enter authentication credentials
s3 = boto3.resource('s3', aws_access_key_id = '******', 
                          aws_secret_access_key= '******')
KEY = 'usdata.json' # replace with your object key
file =s3.Bucket(BUCKET_NAME).download_file(KEY, 'usdata.json')


  

In [3]:

import json
import pandas as pd
from pandas.io.json import json_normalize
with open('usdata.json') as json_data:
    data = json.load(json_data)
df = pd.DataFrame.from_dict(data['data'])

df.head(10)

,ID Nation,Nation,ID Year,Year,Population,Slug Nation
0,01000US,United States,2019,2019,328239523,united-states
1,01000US,United States,2018,2018,327167439,united-states
2,01000US,United States,2017,2017,325719178,united-states
3,01000US,United States,2016,2016,323127515,united-states
4,01000US,United States,2015,2015,321418821,united-states
5,01000US,United States,2014,2014,318857056,united-states
6,01000US,United States,2013,2013,316128839,united-states


In [4]:
df.columns=df.columns.str.lower()
df.columns

Index(['id nation', 'nation', 'id year', 'year', 'population', 'slug nation'], dtype='object')

In [5]:
def std(x): return np.std(x)


In [6]:
import numpy as np
gdf=df.groupby('year')["population"].agg([np.mean, np.std]).fillna(0)
gdf[gdf.index.isin(['2013','2014','2015','2016','2017','2018'])].head(10)

,mean,std
year,,
2013,316128839,0.0
2014,318857056,0.0
2015,321418821,0.0
2016,323127515,0.0
2017,325719178,0.0
2018,327167439,0.0


In [7]:
import os
folder='/pub/time.series/pr/'
my_bucket = s3.Bucket(BUCKET_NAME)
# download file into current directory
objects = my_bucket.objects.filter(Prefix=folder)
for obj in objects:
    path, filename = os.path.split(obj.key)
    my_bucket.download_file(obj.key, filename)


In [8]:
df_series = pd.read_csv('pr.data.0.Current',  sep='\t')
df_series.head(10)

,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.6,NaN
1,PRS30006011,1995,Q02,2.1,NaN
2,PRS30006011,1995,Q03,0.9,NaN
3,PRS30006011,1995,Q04,0.1,NaN
4,PRS30006011,1995,Q05,1.4,NaN
5,PRS30006011,1996,Q01,-0.2,NaN
6,PRS30006011,1996,Q02,-0.3,NaN
7,PRS30006011,1996,Q03,-0.1,NaN
8,PRS30006011,1996,Q04,0.2,NaN
9,PRS30006011,1996,Q05,-0.1,NaN


In [9]:
df_series.columns = df_series.columns.str.strip()
df_series.columns

Index(['series_id', 'year', 'period', 'value', 'footnote_codes'], dtype='object')

In [10]:
max_grp=df_series.loc[df_series.groupby(['series_id','year'])['value'].idxmax()]
max_filter=max_grp[['series_id','year','value']]
max_filter.head(10)

,series_id,year,value
0,PRS30006011,1995,2.6
8,PRS30006011,1996,0.2
13,PRS30006011,1997,1.5
15,PRS30006011,1998,1.6
22,PRS30006011,1999,-1.1
26,PRS30006011,2000,0.3
30,PRS30006011,2001,-1.4
38,PRS30006011,2002,-5.2
40,PRS30006011,2003,-4.4
48,PRS30006011,2004,0.1


In [11]:
#Using both dataframes from Part 1 and Part 2, generate a report that will provide 
#the value for series_id = PRS30006032 and period = Q01 and the population for that
# given year (if available in the population dataset)


series_id	year	period	value	Population


PRS30006032	2018	Q01	1.9	327167439

In [12]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id nation    7 non-null      object
 1   nation       7 non-null      object
 2   id year      7 non-null      int64 
 3   year         7 non-null      object
 4   population   7 non-null      int64 
 5   slug nation  7 non-null      object
dtypes: int64(2), object(4)
memory usage: 464.0+ bytes


In [13]:
df_series.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32409 entries, 0 to 32408
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   series_id       32409 non-null  object 
 1   year            32409 non-null  int64  
 2   period          32409 non-null  object 
 3   value           32409 non-null  float64
 4   footnote_codes  193 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 1.2+ MB


In [14]:
df['year'] = df['year'].astype('int64')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id nation    7 non-null      object
 1   nation       7 non-null      object
 2   id year      7 non-null      int64 
 3   year         7 non-null      int64 
 4   population   7 non-null      int64 
 5   slug nation  7 non-null      object
dtypes: int64(3), object(3)
memory usage: 464.0+ bytes


In [15]:
#series_id = PRS30006032 and period = Q01

inner_merged_total = pd.merge(df_series, df, on=["year"])
inner_merged_total

,series_id,year,period,value,footnote_codes,id nation,nation,id year,population,slug nation
0,PRS30006011,2013,Q01,1.300,NaN,01000US,United States,2013,316128839,united-states
1,PRS30006011,2013,Q02,0.200,NaN,01000US,United States,2013,316128839,united-states
2,PRS30006011,2013,Q03,0.200,NaN,01000US,United States,2013,316128839,united-states
3,PRS30006011,2013,Q04,0.700,NaN,01000US,United States,2013,316128839,united-states
4,PRS30006011,2013,Q05,0.600,NaN,01000US,United States,2013,316128839,united-states
...,...,...,...,...,...,...,...,...,...,...
8605,PRS88003203,2019,Q01,112.294,NaN,01000US,United States,2019,328239523,united-states
8606,PRS88003203,2019,Q02,112.627,NaN,01000US,United States,2019,328239523,united-states
8607,PRS88003203,2019,Q03,113.182,NaN,01000US,United States,2019,328239523,united-states
8608,PRS88003203,2019,Q04,113.421,NaN,01000US,United States,2019,328239523,united-states


In [18]:
dfin=inner_merged_total.convert_dtypes()
dfin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8610 entries, 0 to 8609
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   series_id       8610 non-null   string 
 1   year            8610 non-null   Int64  
 2   period          8610 non-null   string 
 3   value           8610 non-null   float64
 4   footnote_codes  0 non-null      Int64  
 5   id nation       8610 non-null   string 
 6   nation          8610 non-null   string 
 7   id year         8610 non-null   Int64  
 8   population      8610 non-null   Int64  
 9   slug nation     8610 non-null   string 
dtypes: Int64(4), float64(1), string(5)
memory usage: 773.6 KB


In [19]:
any(char.isspace() for char in dfin.iloc[1][0])

True

In [20]:
dfin.iloc[1][0]

'PRS30006011      '

In [21]:
dfin['series_id'] = dfin['series_id'].str.strip()
dfin.iloc[1][0]

'PRS30006011'

In [22]:
dfin.loc[(dfin['series_id']== 'PRS88003203') & (dfin['period']== 'Q01') ][['series_id',	'year',	'period',	'value',	'population']]

,series_id,year,period,value,population
1225,PRS88003203,2013,Q01,100.692,316128839
2455,PRS88003203,2014,Q01,103.532,318857056
3685,PRS88003203,2015,Q01,103.565,321418821
4915,PRS88003203,2016,Q01,104.116,323127515
6145,PRS88003203,2017,Q01,107.832,325719178
7375,PRS88003203,2018,Q01,109.454,327167439
8605,PRS88003203,2019,Q01,112.294,328239523
